In [1]:
import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from warnings import simplefilter
import math
from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split,StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score


## this line is needed sometimes. Sometimes plt.show doesn't work.
%matplotlib inline  
simplefilter('ignore')

c:\Users\Nuwanga Wijamuni\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Nuwanga Wijamuni\AppData\Local\Temp\ipykernel_6232\1183348828.py:7: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [2]:
train_data_1 = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
orig_data = pd.read_csv("WineQT.csv")

In [3]:
print("competition train data set:\n")
train_data_1.info()

competition train data set:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2056 entries, 0 to 2055
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Id                    2056 non-null   int64  
 1   fixed acidity         2056 non-null   float64
 2   volatile acidity      2056 non-null   float64
 3   citric acid           2056 non-null   float64
 4   residual sugar        2056 non-null   float64
 5   chlorides             2056 non-null   float64
 6   free sulfur dioxide   2056 non-null   float64
 7   total sulfur dioxide  2056 non-null   float64
 8   density               2056 non-null   float64
 9   pH                    2056 non-null   float64
 10  sulphates             2056 non-null   float64
 11  alcohol               2056 non-null   float64
 12  quality               2056 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 208.9 KB


In [2]:
pip install pandas_profiling

     ------------------------------------ 324.4/324.4 KB 872.6 kB/s eta 0:00:00
     -------------------------------------- 344.5/344.5 KB 1.5 MB/s eta 0:00:00
     -------------------------------------- 663.4/663.4 KB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 102.7/102.7 KB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 4.7/4.7 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 2.7 MB/s eta 0:00:00
     -------------------------------------- 296.5/296.5 KB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 4.2/4.2 MB 2.5 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27110 sha256=7145b98bb2aaa4cd30d3f60ae6aeecb73513e0ef28bd26c8d567446738c5fece
  Stored in directory: c:\users\n

You should consider upgrading via the 'c:\Users\Nuwanga Wijamuni\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
print("competition test data set:\n")
test_data.info()

competition test data set:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Id                    1372 non-null   int64  
 1   fixed acidity         1372 non-null   float64
 2   volatile acidity      1372 non-null   float64
 3   citric acid           1372 non-null   float64
 4   residual sugar        1372 non-null   float64
 5   chlorides             1372 non-null   float64
 6   free sulfur dioxide   1372 non-null   float64
 7   total sulfur dioxide  1372 non-null   float64
 8   density               1372 non-null   float64
 9   pH                    1372 non-null   float64
 10  sulphates             1372 non-null   float64
 11  alcohol               1372 non-null   float64
dtypes: float64(11), int64(1)
memory usage: 128.8 KB


In [5]:
print("check original data duplicates:\n")
train_data_1.duplicated().sum()

check original data duplicates:



0

In [6]:
print("check train.csv duplicates:\n")
orig_data.duplicated().sum()

check train.csv duplicates:



0

In [7]:
# Combined train.csv and WineQT.csv into one train data set: train_data
train_data = pd.concat([train_data_1,orig_data]).reset_index(drop=True)

In [8]:
# Experimentation: Adding new features.


train_data['total_acid'] = train_data['fixed acidity'] + train_data['volatile acidity'] + train_data['citric acid']
train_data['acid/density'] = train_data['total_acid']  / train_data['density']
train_data['alcohol_density'] = train_data['alcohol']  * train_data['density']
train_data['sulphate/density'] = train_data['sulphates']  / train_data['density']
train_data['sulphates/acid'] = train_data['sulphates'] / train_data['volatile acidity']
train_data['sulphates/chlorides'] = train_data['sulphates'] / train_data['chlorides']
train_data['sulphates*alcohol'] = train_data['sulphates'] * train_data['alcohol']

In [10]:
train_data.head(10)

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,total_acid,acid/density,alcohol_density,sulphate/density,sulphates/acid,sulphates/chlorides,sulphates*alcohol
0,0,8.0,0.50,0.39,2.2,0.073,30.0,39.0,0.99572,3.33,0.77,12.1,6,8.89,8.928213,12.048212,0.773310,1.540000,10.547945,9.317
1,1,9.3,0.30,0.73,2.3,0.092,30.0,67.0,0.99854,3.32,0.67,12.8,6,10.33,10.345104,12.781312,0.670980,2.233333,7.282609,8.576
2,2,7.1,0.51,0.03,2.1,0.059,3.0,12.0,0.99660,3.52,0.73,11.3,7,7.64,7.666065,11.261580,0.732490,1.431373,12.372881,8.249
3,3,8.1,0.87,0.22,2.6,0.084,11.0,65.0,0.99730,3.20,0.53,9.8,5,9.19,9.214880,9.773540,0.531435,0.609195,6.309524,5.194
4,4,8.5,0.36,0.30,2.3,0.079,10.0,45.0,0.99444,3.20,1.36,9.5,6,9.16,9.211214,9.447180,1.367604,3.777778,17.215190,12.920
5,5,9.9,0.51,0.44,2.2,0.111,30.0,134.0,0.99820,3.11,0.54,9.6,5,10.85,10.869565,9.582720,0.540974,1.058824,4.864865,5.184
6,6,7.2,0.87,0.00,2.3,0.080,6.0,18.0,0.99552,3.34,0.60,11.3,6,8.07,8.106316,11.249376,0.602700,0.689655,7.500000,6.780
7,7,7.5,0.43,0.32,1.8,0.066,18.0,40.0,0.99560,3.30,0.43,9.7,6,8.25,8.286460,9.657320,0.431900,1.000000,6.515152,4.171
8,8,11.6,0.38,0.55,2.2,0.084,17.0,40.0,1.00080,3.17,0.73,9.8,6,12.53,12.519984,9.807840,0.729416,1.921053,8.690476,7.154
9,9,7.8,0.78,0.09,2.2,0.049,13.0,29.0,0.99682,3.51,0.49,9.5,5,8.67,8.697659,9.469790,0.491563,0.628205,10.000000,4.655


In [11]:
train_data.dtypes.unique

<bound method Series.unique of Id                        int64
fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
total_acid              float64
acid/density            float64
alcohol_density         float64
sulphate/density        float64
sulphates/acid          float64
sulphates/chlorides     float64
sulphates*alcohol       float64
dtype: object>

In [12]:
test_data['total_acid'] = test_data['fixed acidity'] + test_data['volatile acidity'] + test_data['citric acid']
test_data['acid/density'] = test_data['total_acid']  / test_data['density']
test_data['alcohol_density'] = test_data['alcohol']  * test_data['density']
test_data['sulphate/density'] = test_data['sulphates']  / test_data['density']
test_data['sulphates/acid'] = test_data['sulphates'] / test_data['volatile acidity']
test_data['sulphates/chlorides'] = test_data['sulphates'] / test_data['chlorides']
test_data['sulphates*alcohol'] = test_data['sulphates'] * test_data['alcohol']

In [13]:
test_data.shape

(1372, 19)

In [14]:
# Let's scale all the features using Scaler

# set the features
orig_features=["fixed acidity", "volatile acidity", "citric acid","residual sugar",
            "chlorides","free sulfur dioxide","total sulfur dioxide",
               "density","pH","sulphates","alcohol"]

new_features=['total_acid','acid/density','alcohol_density','sulphate/density',
             'sulphates/acid','sulphates/chlorides','sulphates*alcohol']

wine_features = orig_features + new_features

#wine_features = orig_features

from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.preprocessing import MinMaxScaler

# create scaler
#scaler = StandardScaler() ## (Standardization)if the distribution is Gaussian
#scaler = MinMaxScaler() ## (Normalization) if the distribution is not Gaussian or unknown
scaler = QuantileTransformer(output_distribution="normal")

# transform the features

X_train_full = train_data[wine_features]
X_test_full = test_data[wine_features]

scaled_X_train = pd.DataFrame(scaler.fit_transform(X_train_full))
scaled_X_test = pd.DataFrame(scaler.transform(X_test_full))


# scaling removed column names; put them back
scaled_X_train.columns = X_train_full.columns
scaled_X_test.columns = X_test_full.columns

In [15]:
wine_features

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'total_acid',
 'acid/density',
 'alcohol_density',
 'sulphate/density',
 'sulphates/acid',
 'sulphates/chlorides',
 'sulphates*alcohol']

In [16]:
# target
# 'quality' is a ordinal variable. 
y = train_data.quality - 3 

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(scaled_X_train, y, 
                                                      test_size = 0.2, 
                                                      random_state = 0)



#model and training
simple_xgb = XGBClassifier(learning_rate = 0.05, 
                           n_estimators=1000,
                           seed=0)


#simple_xgb = XGBClassifier(tree_method = 'hist',
#                               colsample_bytree = 0.75, 
#                               gamma = 0.8,  
#                               max_depth = 15, 
#                               min_child_weight = 1,
#                               n_estimators=1000,
#                               seed=0,
#                               learning_rate=0.05)

#my_model.fit(X_train_full,y)
simple_xgb.fit(X_train,y_train,early_stopping_rounds=5,
            eval_set=[(X_valid,y_valid)],verbose=False)


## Predicting on X_valid and test data
simple_xgb_pred_1 = simple_xgb.predict(X_valid)
    
        

## Computing roc-auc score
simple_xgb_cv_scores = cohen_kappa_score(y_valid, simple_xgb_pred_1, 
                                         weights = 'quadratic')
   
print('The roc-auc score from simple split is:', simple_xgb_cv_scores)

The roc-auc score from simple split is: 0.5589331948017127


In [18]:
XGB_simple_sub = simple_xgb.predict(scaled_X_test) + 3

In [19]:
# output predictions data
output = pd.DataFrame({'Id': test_data.Id, 'quality': XGB_simple_sub})
output.to_csv('submission_xgb_simple.csv', index=False)

In [20]:
!head submission_xgb_simple.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


In [21]:


X = scaled_X_train
test_md = scaled_X_test

XGB_cv_scores, XGB_imp = list(), list()


## Running 5 times CV
for i in range(5):
    
    skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
        
        ## Building RF model
        XGB_md = XGBClassifier(tree_method = 'hist',
                               colsample_bytree = 0.75, 
                               gamma = 0.8,  
                               max_depth = 10, 
                               min_child_weight = 15,
                               n_estimators=1000,
                               seed=0,
                               learning_rate=0.05)
    
        ## Train the model:
    
        XGB_md.fit(X_train,y_train,early_stopping_rounds=5,
            eval_set=[(X_test,y_test)],verbose=False)
        
        
        
        ## Predicting on X_test and test data
        XGB_pred_1 = XGB_md.predict(X_test)
    
        
        ## Computing roc-auc score
        XGB_cv_scores.append(cohen_kappa_score(y_test, XGB_pred_1, weights = 'quadratic'))
        

XGB_cv_score = np.mean(XGB_cv_scores)    
print('The average roc-auc score over 5-folds (run 5 times) is:', XGB_cv_score)

The average roc-auc score over 5-folds (run 5 times) is: 0.5172225065122117


In [22]:
XGB_md_sub = XGB_md.predict(scaled_X_test) + 3

In [23]:
# output predictions data
output = pd.DataFrame({'Id': test_data.Id, 'quality': XGB_md_sub})
output.to_csv('submission_xgb_skfold.csv', index=False)

In [24]:
!head submission_xgb_skfold.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


In [25]:
d = {
    'test_size': ['0.2', '0.3'],
    'roc-auc score (with new fea.)': ['0.559', '0.509' ],
    'roc-auc score(without new fea.)': ['0.525', '0.512']
}
df = pd.DataFrame(data=d)
df.style.set_caption("XGB + Simple test_split")

,test_size,roc-auc score (with new fea.),roc-auc score(without new fea.)
0,0.2,0.559,0.525
1,0.3,0.509,0.512
